In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

def train_predict_student_score(student_id, effort_hours):
    conn = sqlite3.connect("database.db")
    performance_data = pd.read_sql_query("SELECT * FROM Performance", conn)
    student_performance_data = performance_data[performance_data["StudentID"] == student_id]
    x_data = student_performance_data[["effort_hours"]]
    y_data = student_performance_data[["marks"]]

    model = LinearRegression()
    model.fit(x_data, y_data)

    x_test = pd.DataFrame({"effort hours": [effort_hours]})
    prediction = model.predict(x_test)

    plt.scatter(x_data, y_data, color="blue", label="Performance Data")
    plt.plot(x_data, model.predict(x_data), color="red", label="Regression Line")
    plt.xlabel("Effort Hours")
    plt.ylabel("Marks")
    plt.title(f"Linear Regression for Student  {student_id}")
    plt.legend()
    plt.show()

    return prediction[0]

students = ["SID20131151", "SID20149500", "SID20182516"]
for student_id in students:
    predicted_score =  train_predict_student_score(student_id, effort_hours=10)
    print(f"Predicted score for student {student_id} with 10 hours of effort: {predicted_score}")